## Preprocessing data of Markenscoff-Papadimitriou et al., Cell (2020)

In [3]:
import pandas as pd
import os, glob
from pybedtools import BedTool

In [17]:
pre = pd.read_excel("~/Dropbox/CWAS_paper_WD/Data/raw_annotations/Markenscoff-Papadimitriou_2020_TableS2.xlsx", sheet_name="S2A")
pre = pre.loc[pre['pre_hg38'].notna()]

In [18]:
regions_peak = ['overlaps_cge_peak',
 'overlaps_lge_peak',
 'overlaps_mge_peak',
 'overlaps_motor_peak',
 'overlaps_parietal_peak',
 'overlaps_pfc_peak',
 'overlaps_s1_peak',
 'overlaps_temporal_peak',
 'overlaps_v1_peak']

In [19]:
bg_regions = regions_peak[:3]
cortex_regions = regions_peak[3:]

pre_bg = pre.loc[:, ["pre_hg38"]+bg_regions]
pre_bg['bg_overlaps'] = pre_bg.iloc[:, 1:].sum(axis=1)
pre_bg = pre_bg.loc[pre_bg['bg_overlaps']>0]

pre_bg_bed = pre_bg['pre_hg38'].str.split(':|-', expand=True)
pre_bg_bed.columns = ['chr','start','end']
pre_bg_bed = pre_bg_bed.loc[(pre_bg_bed['chr'].notna())&(pre_bg_bed['start'].notna())&(pre_bg_bed['end'].notna())]
pre_bg_bed = BedTool.from_dataframe(pre_bg_bed).sort().to_dataframe()
pre_bg_bed.to_csv("~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_basal_ganglia_enhancers.hg38.sorted.bed", sep="\t", index=False, header=False)
os.system(f"bgzip -c ~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_basal_ganglia_enhancers.hg38.sorted.bed > ~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_basal_ganglia_enhancers.hg38.sorted.bed.gz")

0

In [20]:

pre_cortex = pre.loc[:, ["pre_hg38"]+cortex_regions]
pre_cortex['cortex_overlaps'] = pre_cortex.iloc[:, 1:].sum(axis=1)
pre_cortex = pre_cortex.loc[pre_cortex['cortex_overlaps']>0]

pre_cortex_bed = pre_cortex['pre_hg38'].str.split(':|-', expand=True)
pre_cortex_bed.columns = ['chr','start','end']
pre_cortex_bed = pre_cortex_bed.loc[(pre_cortex_bed['chr'].notna())&(pre_cortex_bed['start'].notna())&(pre_cortex_bed['end'].notna())]
pre_cortex_bed = BedTool.from_dataframe(pre_cortex_bed).sort().to_dataframe()
pre_cortex_bed.to_csv("~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_cortex_enhancers.hg38.sorted.bed", sep="\t", index=False, header=False)
os.system(f"bgzip -c ~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_cortex_enhancers.hg38.sorted.bed > ~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_cortex_enhancers.hg38.sorted.bed.gz")

0

In [14]:
os.system(f"rm ~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_cortex_enhancers.hg38.sorted.bed ~/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_basal_ganglia_enhancers.hg38.sorted.bed")

0

## Create index file

In [15]:
all_files = glob.glob("/Users/yujinkim/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/*.bed.gz")
len(all_files)

67

In [16]:
pre = [z for z in all_files if os.path.basename(z).startswith("pRE_")]
pre

['/Users/yujinkim/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_basal_ganglia_enhancers.hg38.sorted.bed.gz',
 '/Users/yujinkim/Dropbox/CWAS_paper_WD/Data/cwas_annotation.v5/pRE_cortex_enhancers.hg38.sorted.bed.gz']

In [17]:
for f in pre:
    os.system(f"tabix -p bed {f} -f")